##### Copyright 2019 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Keras를 이용한 모델의 저장 및 직렬화

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/guide/keras/save_and_serialize"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/keras/save_and_serialize.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/guide/keras/save_and_serialize.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/guide/keras/save_and_serialize.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

이 가이드의 첫 번째 파트에서는 순차 모델 및 함수형 API를 사용하여 제작된 모델에 대한 저장 및 직렬화를 다룹니다. 저장 및 직렬화 API는 이 두 유형의 모델 모두에서 동일합니다.

사용자 정의 하위 클래스 `모델`을 저장하는 방법은 "하위 클래스 모델 저장하기" 섹션에서 다룹니다. 이 경우 API는 순차 또는 함수형 모델과는 약간 다릅니다.

## 설치

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version은 Colab에서만 지원합니다.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

tf.keras.backend.clear_session()  # notebook 상태를 빠르게 초기화 합니다.

## Part I: 순차 모델 또는 함수형 모델 저장하기

다음 모델을 보세요:

In [0]:
from tensorflow import keras
from tensorflow.keras import layers

inputs = keras.Input(shape=(784,), name='digits')
x = layers.Dense(64, activation='relu', name='dense_1')(inputs)
x = layers.Dense(64, activation='relu', name='dense_2')(x)
outputs = layers.Dense(10, activation='softmax', name='predictions')(x)

model = keras.Model(inputs=inputs, outputs=outputs, name='3_layer_mlp')
model.summary()

모델을 훈련합니다. 이제 이 모델은 저장할 데이터인 옵티마이저 상태와 가중치를 포함하고 있습니다.
물론 훈련이 되지 않은 모델도 저장할 수 있습니다.

In [0]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train.reshape(60000, 784).astype('float32') / 255
x_test = x_test.reshape(10000, 784).astype('float32') / 255

model.compile(loss='sparse_categorical_crossentropy',
              optimizer=keras.optimizers.RMSprop())
history = model.fit(x_train, y_train,
                    batch_size=64,
                    epochs=1)

In [0]:
# predictions은 나중에 사용하기 때문에 저장합니다.
predictions = model.predict(x_test)


### 전체 모델 저장하기

함수형 API로 생성된 모델을 단일 파일로 저장할 수 있습니다. 모델을 만든 코드가 없는 경우에도 나중에 이 파일을 사용해서 동일한 모델을 재생성할 수 있습니다.

이 파일은 다음 내용을 포함하고 있습니다:

- 모델의 아키텍처
- 모델의 가중치 (훈련으로 학습된 가중치)
- 모델의 훈련 config 값
- 옵티마이저와 그 상태(훈련이 중단된 곳에서 다시 훈련을 재개할 수 있습니다.)

In [0]:
# 모델을 저장합니다.
model.save('path_to_my_model.h5')

# 파일로부터 동일한 모델을 재생성합니다.
new_model = keras.models.load_model('path_to_my_model.h5')

In [0]:
import numpy as np

# 모델의 상태가 보존되었는지 확인합니다.
new_predictions = new_model.predict(x_test)
np.testing.assert_allclose(predictions, new_predictions, rtol=1e-6, atol=1e-6)

# optimizer 상태가 보존되었는지 확인합니다.:
# 훈련이 중단된 곳에서 다시 훈련을 재개할 수 있습니다.

### SavedModel로 내보내기

전체 모델을 TensorFlow`SavedModel` 형식으로 저장할 수도 있습니다. TensorFlow가 지원하는 `SavedModel`은 TensorFlow 객체에 독립적인 직렬화 형식으로 Python가 아닌 TensorFlow 구현도 지원합니다.

In [0]:
# 모델을 SavedModel로 저장합니다.
model.save('path_to_saved_model', save_format='tf')

# 동일한 모델을 재생성합니다.
new_model = keras.models.load_model('path_to_saved_model')

# 모델의 상태가 보존되었는지 확인합니다.
new_predictions = new_model.predict(x_test)
np.testing.assert_allclose(predictions, new_predictions, rtol=1e-6, atol=1e-6)

# optimizer 상태가 보존되었는지 확인:
# 훈련이 중단된 곳에서 다시 훈련을 재개할 수 있습니다.

생성된 `SavedModel` 은 다음 내용을 포함하고 있습니다:

- 모델의 가중치를 포함하는 checkpoint
- 기본 TensorFlow graph를 포함하는 `SavedModel` 프로토.

### 아키텍처만 저장하기

때로는 모델의 아키텍처에만 관심이 있으며 가중치나 옵티마이저를 저장할 필요가 없습니다. 이 경우 `get_config()` 함수를 통해 모델의 "config"를 검색 할 수 있습니다. 이 config값은 파이썬의 딕셔너리로, 이전의 훈련을 통해 학습하지 않아도 처음부터 초기화 된 동일한 모델을 다시 만들 수 있습니다.

In [0]:
config = model.get_config()
reinitialized_model = keras.Model.from_config(config)

# 모델의 상태는 보존되지 않습니다! 모델의 아키텍처만 저장했습니다.
new_predictions = reinitialized_model.predict(x_test)
assert abs(np.sum(predictions - new_predictions)) > 0.

`from_json ()`의 대안으로 `to_json ()`을 사용할 수 있습니다. 이 함수는 파이썬 딕셔너리 대신 JSON 문자열을 사용하여 config값을 저장합니다. 이 함수는 config값을 디스크에 저장하는 데 유용합니다.

In [0]:
json_config = model.to_json()
reinitialized_model = keras.models.model_from_json(json_config)

### 가중치만 저장하기

때로는 모델의 상태 (가중치 값)에만 관심이 있고 아키텍처에는 관심이 없는 경우가 있습니다. 이 경우 `get_weights ()`를 통해 Numpy 배열의 리스트로 가중치 값을 가져오고, `set_weights`를 통해 모델의 상태를 설정할 수 있습니다.:

In [0]:
weights = model.get_weights()  # 모델의 상태를 가져옵니다..
model.set_weights(weights)  # 모델의 상태를 설정합니다.

`get_config ()`/`from_config ()` 및 `get_weights ()`/`set_weights ()`를 결합하여 모델을 동일한 상태로 다시 만들 수 있습니다. 그러나 'model.save ()'와 달리 훈련 config 및 옵티마이저는 포함되지 않습니다. 모델을 훈련하기 전에 `compile()`을 다시 호출해야 합니다.

In [0]:
config = model.get_config()
weights = model.get_weights()

new_model = keras.Model.from_config(config)
new_model.set_weights(weights)

# 상태가 보존되었는지 확인하세요.
new_predictions = new_model.predict(x_test)
np.testing.assert_allclose(predictions, new_predictions, rtol=1e-6, atol=1e-6)

# 옵티마이저가 보존되지 않았다는 것을 확인하세요,
# 따라서 훈련 전에 모델을 새로 컴파일해야합니다
# (그리고 옵티마이저는 빈 상태에서 시작합니다).

디스크에 저장하는 방법으로는 `get_weights()` 와 `set_weights(weights)` 대신
`save_weights(fpath)` 와 `load_weights(fpath)`가 있습니다.

아래는 디스크에 저장하는 예제입니다:

In [0]:
# JSON 형식의 config값을 디스크로 저장합니다.
json_config = model.to_json()
with open('model_config.json', 'w') as json_file:
    json_file.write(json_config)
# 가중치를 디스크에 저장합니다.
model.save_weights('path_to_my_weights.h5')

# 저장했던 2개의 파일에서 모델을 다시 가져옵니다.
with open('model_config.json') as json_file:
    json_config = json_file.read()
new_model = keras.models.model_from_json(json_config)
new_model.load_weights('path_to_my_weights.h5')

# 모델의 상태가 보존되었는지 확인하세요.
new_predictions = new_model.predict(x_test)
np.testing.assert_allclose(predictions, new_predictions, rtol=1e-6, atol=1e-6)

# 옵티마이저는 보존되지 않았습니다.

그러나 가장 간단하고 권장되는 방법은 바로 이것입니다:

In [0]:
model.save('path_to_my_model.h5')
del model
model = keras.models.load_model('path_to_my_model.h5')

### 텐서플로 checkpoints를 사용해 가중치만 저장하기

`save_weights`은 Keras HDF5 형식 또는 텐서플로 Checkpoint 형식으로 파일을 작성할 수 있습니다.
[텐서플로 Checkpoint format](https://www.tensorflow.org/api_docs/python/tf/train/Checkpoint). 형식은 제공한 파일 확장자에서 유추됩니다. ".h5"또는 ".keras"인 경우 프레임 워크는 Keras HDF5 형식을 사용합니다. 다른 것은 기본적으로 Checkpoint입니다.

In [0]:
model.save_weights('path_to_my_tf_checkpoint')

정확히 말하자면, 형식은 `save_format`매개변수를 통해 명시적으로 전달 될 수 있으며, "tf"또는 "h5"값을 가질 수 있습니다.:

In [0]:
model.save_weights('path_to_my_tf_checkpoint', save_format='tf')

## 서브 클래스 모델 저장하기

순차 모델 및 함수형 모델은 계층의 DAG를 나타내는 데이터 구조입니다. 따라서,
안전하게 직렬화 및 직렬화 해제 할 수 있습니다.

서브 클래스된 모델은 데이터 구조가 아니라 코드 조각이라는 점이 다릅니다. 모델의 아키텍처는
`call` 함수의 본문을 통해 정의됩니다. 이는 모델의 아키텍처를 안전하게 직렬화 할 수 없음을 의미합니다. 모델을 로드하려면 모델을 생성한 코드(모델 서브 클래스의 코드)에 액세스 할 수 있어야 합니다. 또는 이 코드를 바이트 코드로 직렬화 할 수 있지만 (예 : python pickle 모듈을 통해) 안전하지 않으며 일반적으로 이식성이 없습니다.

이러한 차이점에 대한 자세한 정보는 ["TensorFlow 2.0의 기호 및 명령형 API 란 무엇입니까?"](https://medium.com/tensorflow/what-are-symbolic-and-imperative-apis-in-tensorflow-2-0-dfccecb01021) 기사를 참조하십시오.

첫 번째 섹션의 모델과 동일한 구조를 따르는 다음 서브 클래스 모델을 고려해 봅시다.:

In [0]:
class ThreeLayerMLP(keras.Model):

  def __init__(self, name=None):
    super(ThreeLayerMLP, self).__init__(name=name)
    self.dense_1 = layers.Dense(64, activation='relu', name='dense_1')
    self.dense_2 = layers.Dense(64, activation='relu', name='dense_2')
    self.pred_layer = layers.Dense(10, activation='softmax', name='predictions')

  def call(self, inputs):
    x = self.dense_1(inputs)
    x = self.dense_2(x)
    return self.pred_layer(x)

def get_model():
  return ThreeLayerMLP(name='3_layer_mlp')

model = get_model()

우선, * 사용 된 적이 없는 서브 클래스 모델은 저장할 수 없습니다 *.

왜냐하면 일부 데이터에서 가중치를 생성하기 위해 서브 클래스 모델을 호출해야 하기 때문입니다.

모델이 호출 될 때, 예상되는 입력 데이터의 shape과 데이터 타입을 알지 못하므로 가중치 변수를 만들 수 없습니다.
앞서 첫 번째 섹션의 함수형 모델에서는 입력의 shape과 데이터 타입을 미리 정했다는 것을 기억하실 겁니다. (`keras.Input (...)`을 통해). 따라서 함수형 모델은 인스턴스화 되자마자 바로 상태를 갖습니다.

모델을 훈련하여 상태를 가지도록 합니다:

In [0]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train.reshape(60000, 784).astype('float32') / 255
x_test = x_test.reshape(10000, 784).astype('float32') / 255

model.compile(loss='sparse_categorical_crossentropy',
              optimizer=keras.optimizers.RMSprop())
history = model.fit(x_train, y_train,
                    batch_size=64,
                    epochs=1)

서브 클래스 모델을 저장하는 권장 방법은 `save_weights`를 사용하여 모델과 관련된 모든 변수의 값을 포함하는 TensorFlow SavedModel 체크 포인트를 생성하는 것입니다.
- 레이어의 가중치
- 옵티마이저의 상태
- 상태가 저장된 모델의 metric과 관련된 모든 변수 (있는 경우)



In [0]:
model.save_weights('path_to_my_weights', save_format='tf')

In [0]:
# predictions은 나중에 사용하기 때문에 저장합니다.
predictions = model.predict(x_test)
# 첫 번째 배치에서의 손실도 저장합니다.
# 나중에 옵티마이저 상태가 보존되었다는 것을 확인하기 위해 저장합니다.
first_batch_loss = model.train_on_batch(x_train[:64], y_train[:64])

모델을 복원하려면 모델 객체를 생성한 코드에 액세스 해야 합니다.

옵티마이저 상태 및 상태가 저장된 metric의 상태를 복원하려면 모델을 컴파일하고 (이전과 동일한 매개변수로)
`load_weights`을 호출하기 전에 일부 데이터를 호출해야합니다.:

In [0]:
# 모델을 다시 만듭니다.
new_model = get_model()
new_model.compile(loss='sparse_categorical_crossentropy',
                  optimizer=keras.optimizers.RMSprop())

# 상태가 저장된 metric 변수 뿐만 아니라
# 옵티마이저가 사용하는 변수를 초기화 합니다.
new_model.train_on_batch(x_train[:1], y_train[:1])

# 이전 모델의 상태를 가져옵니다.
new_model.load_weights('path_to_my_weights')

# 모델의 상태가 보존되었는지 확인합니다.
new_predictions = new_model.predict(x_test)
np.testing.assert_allclose(predictions, new_predictions, rtol=1e-6, atol=1e-6)

# 옵티마이저의 상태도 보존됩니다.
# 중단한 곳에서 훈련을 재개 할 수 있습니다.
new_first_batch_loss = new_model.train_on_batch(x_train[:64], y_train[:64])
assert first_batch_loss == new_first_batch_loss

이 안내서의 끝에 도달했습니다! 여기에는 TensorFlow 2.0의 tf.keras로 모델을 저장하고 직렬화 하는데 필요한 모든 내용이 포함됩니다.